In [1]:
import my_setting.utils as utils
import cufflinks as cf
import jieba
import matplotlib.pyplot as plt
import missingno as msno
import pandas as pd
import seaborn as sns

from wordcloud import WordCloud

%matplotlib inline
cf.go_offline()
pd.set_option('display.max_columns', None)
pd.set_option('max_colwidth',100)
plt.style.use('seaborn')


In [2]:
output_900k=pd.read_csv('/home/zhw/PycharmProjects/nCovSentimentAnalysis/Output/submit_file.csv',encoding='utf-8')
output_900k.head(10)

,id,y
0,4456074364642450,0
1,4456069114155340,0
2,4456068145061890,0
3,4456064403586410,0
4,4456060284446840,0
5,4456059202748560,0
6,4456055234844520,0
7,4456071538097160,0
8,4456071366149900,0
9,4456070334552490,0


In [3]:
temp=output_900k['y'].value_counts()
temp_df=pd.DataFrame({'labels':temp.index,'values':temp.values})
temp_df.iplot(kind='pie',labels='labels',values='values',title='Labels')

In [ ]:
simplify_weibo=pd.read_csv('/home/zhw/PycharmProjects/nCovSentimentAnalysis/Data/UTF8nCoV_900k_train.unlabled.csv',encoding='utf-8')
simplify_weibo.head(10)

In [ ]:
simplify_weibo.isnull().any()

In [ ]:
msno.bar(simplify_weibo)

In [ ]:
temp=simplify_weibo['label'].value_counts()
temp_df=pd.DataFrame({'labels':temp.index,'values':temp.values})
temp_df.iplot(kind='pie',labels='labels',values='values',title='Labels')

In [ ]:
simplify_weibo['length']=simplify_weibo['review'].astype(str).apply(len)

In [ ]:
fig,ax1=plt.subplots(1,figsize=(14,8))
sns.distplot(simplify_weibo['length'],ax=ax1,color='green')
plt.rcParams['font.family']='STSong'
plt.rcParams['axes.unicode_minus']=False
ax1.set_title('长度分布')

plt.show()

In [ ]:
nip_corpus=pd.read_csv('/home/zhw/PycharmProjects/nCovSentimentAnalysis/Data/SentimentRelevantCorpus/unzip/chineseNIP_weibo_senti_100k.csv',encoding='utf-8')
nip_corpus.head(10)

In [ ]:
nip_corpus.isnull().any()

In [ ]:
msno.bar(nip_corpus)

In [ ]:
temp = nip_corpus['label'].value_counts()
temp_df = pd.DataFrame({'labels':temp.index,'values':temp.values})
temp_df.iplot(kind='pie',labels='labels',values='values',title='Labels')

In [ ]:
nip_corpus['length']=nip_corpus['review'].astype(str).apply(len)

In [ ]:
fig, ax1 = plt.subplots(1,figsize=(14,8))
sns.distplot(nip_corpus['length'],ax=ax1, color='green')
plt.rcParams['font.family']='SIMHEI' 
plt.rcParams['axes.unicode_minus'] = False
ax1.set_title('训练集微博长度分布')

plt.show()


## 读取数据

由于其他编码会出现少部分汉字乱码，比如“XX超话”、表情符号等。所以我手动用记事本转换成了 UTF-8 编码。

其中，超话以及其他特殊字符以“”表示，大部分表情符号被“??”代替。

In [ ]:
train_labeled = pd.read_csv('/home/zhw/PycharmProjects/nCovSentimentAnalysis/Data/UTF8nCoV_100k_train.labled.csv', encoding='utf-8')
train_labeled.rename(columns = {"微博id": "Weibo_ID",
                                "微博发布时间": "Publish_Time", 
                                "发布人账号": "Account_ID",
                                "微博中文内容": "Chinese_Content",
                                "微博图片": "Pictures",
                                "微博视频": "Videos",
                                "情感倾向": "Labels"},  inplace=True)

train_labeled_copy = train_labeled.copy()

In [ ]:
test = pd.read_csv('/home/zhw/PycharmProjects/nCovSentimentAnalysis/Data/UTF8nCov_10k_test.csv', encoding='utf-8')
test.rename(columns = {"微博id": "Weibo_ID",
                                "微博发布时间": "Publish_Time", 
                                "发布人账号": "Account_ID",
                                "微博中文内容": "Chinese_Content",
                                "微博图片": "Pictures",
                                "微博视频": "Videos"},  inplace=True)

test_copy = test.copy()

In [ ]:
train_labeled.head(5)

In [ ]:
test.head(5)

## 缺失值检查

检查一下训练集和测试集的缺失值，可以看出有些数据是缺失的。

In [ ]:
train_labeled.isnull().any()

In [ ]:
msno.bar(train_labeled)

In [ ]:
msno.bar(test)

有少量微博正文数据丢失，部分labels为空。

## 标签检查

标签统计如下：

In [ ]:
train_labeled_copy.fillna({"Labels": "empty_label"}, inplace=True)

temp = train_labeled_copy["Labels"].value_counts()
temp_df = pd.DataFrame({'labels': temp.index,
                        'values': temp.values})

temp_df.iplot(kind='pie',labels='labels',values='values', title='Labels')

有6种噪音标签，每种各一个，此外还有一点点空标签，统计如下:

In [ ]:
noise = train_labeled_copy[(train_labeled_copy["Labels"] != '0') & 
                           (train_labeled_copy["Labels"] != '1') & 
                           (train_labeled_copy["Labels"] != '-1')]

noise = noise["Labels"].value_counts()
noise_df = pd.DataFrame({'labels': noise.index,
                         'values': noise.values})

noise_df.iplot(kind='pie',labels='labels',values='values', title='奇怪标签统计')

对于非正常标签的处理，可以直接舍弃，也可以手工打上真正标签（如果不嫌累）。

这里我们直接舍弃就行。

In [ ]:
train_labeled_copy['time'] = pd.to_datetime('2020年' + train_labeled['Publish_Time'], format='%Y年%m月%d日 %H:%M', errors='ignore')
test_copy['time'] = pd.to_datetime('2020年' + train_labeled['Publish_Time'], format='%Y年%m月%d日 %H:%M', errors='ignore')

In [ ]:
train_labeled_copy['month'] =  train_labeled_copy['time'].dt.month
train_labeled_copy['day'] =  train_labeled_copy['time'].dt.day
train_labeled_copy['dayfromzero']  = (train_labeled_copy['month'] - 1) * 31 +  train_labeled_copy['day']

test_copy['month'] =  test_copy['time'].dt.month
test_copy['day'] =  test_copy['time'].dt.day
test_copy['dayfromzero']  = (test_copy['month'] - 1) * 31 +  test_copy['day']

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(16, 8))

sns.kdeplot(train_labeled_copy.loc[train_labeled_copy['Labels'] == '0', 'dayfromzero'], ax=ax[0], label='sent(0)')
sns.kdeplot(train_labeled_copy.loc[train_labeled_copy['Labels'] == '1', 'dayfromzero'], ax=ax[0], label='sent(1)')
sns.kdeplot(train_labeled_copy.loc[train_labeled_copy['Labels'] == '-1', 'dayfromzero'], ax=ax[0], label='sent(-1)')

train_labeled_copy.loc[train_labeled_copy['Labels'] == '0', 'dayfromzero'].hist(ax=ax[1])
train_labeled_copy.loc[train_labeled_copy['Labels'] == '1', 'dayfromzero'].hist(ax=ax[1])
train_labeled_copy.loc[train_labeled_copy['Labels'] == '-1', 'dayfromzero'].hist(ax=ax[1])
ax[1].legend(['sent(0)', 'sent(1)','sent(-1)'])

plt.show()

可以看出如下情况，顺便也帮大家找了找相关新闻节点（深藏功与名）：

- **1月18日**后，话题量有明显增长。
    - 1月19日：武汉CDC李刚：新冠人传人风险较低，传染力不强。
    - 1月20日：钟南山:新型肺炎存在人传人现象。
    - 1月20日：口罩出现抢购现象。
    - 1月23日：武汉封城。
    - 1月25日：火神山医院设计方案完成；雷神山医院建造决定。

- **二月九日**前后，话题量达到顶峰。
    - 2月7日：“吹哨人”李文亮医生不幸去世。
    - 2月10日：湖北省相关领导任免。

18日前官方口径（包括网友）们的态度还比较乐观，大部分认为“新冠可防可控”、“传染力有限”，然后钟南山院士对新冠“肯定存在人传人”的表态可能是导致话题量飙升的重要原因。

7日晚上李文亮医生不幸去世，微博和朋友圈都在刷屏，从曲线上看，相关话题量几乎到达顶峰。

（**悼念李文亮医生**）

## 正文长度统计

现在开始统计相关微博的长度，训练集和测试集都有。

In [ ]:
train_labeled_copy['Chinese_Content_Length'] = train_labeled['Chinese_Content'].astype(str).apply(len)
test_copy['Chinese_Content_Length'] = train_labeled['Chinese_Content'].astype(str).apply(len)

In [ ]:
fig, (ax1, ax2) = plt.subplots(1,2,figsize=(14,8))
sns.distplot(train_labeled_copy['Chinese_Content_Length'], ax=ax1, color='blue')
sns.distplot(test_copy['Chinese_Content_Length'], ax=ax2, color='green')

plt.rcParams['font.sans-serif']=['SimHei']
plt.rcParams['axes.unicode_minus'] = False

ax1.set_title('训练集微博长度分布')
ax2.set_title('测试集微博长度分布')

plt.show()
### 正文词云

采用 `jieba` 和 `wordcloud` 对正文做一个词云。

In [ ]:
stop = open('/home/zhw/PycharmProjects/nCovSentimentAnalysis/Docs/cn_stopwords.txt', 'r+', encoding='utf-8')
stopword = stop.read().split("\n")
stopeword = set(stopword)
stop.close()

In [ ]:
def stripword(seg):
    """停用词处理"""
    wordlist = []
    
    for key in seg.split(' '):
        #去除停用词和单字
        if not (key.strip() in stopword) and (len(key.strip()) > 1):
            wordlist.append(key)
    return ' '.join(wordlist)

def cutword(content):
    """分词，去除停用词，写得比较简陋"""
    seg_list = jieba.cut(content)
    line = " ".join(seg_list)
    word = stripword(line)
    return word

In [ ]:
train_labeled_copy['Chinese_Content_cut'] = train_labeled['Chinese_Content'].astype(str).apply(cutword)

In [ ]:
train_labeled_copy.head(3)

In [ ]:
font = r'/home/zhw/PycharmProjects/nCovSentimentAnalysis/Docs/MYSH.TTC'
wc = WordCloud(font_path=font,
               max_words=2000,
               width=1800, 
               height=1600, 
               mode='RGBA', 
               background_color=None).generate(str(train_labeled_copy['Chinese_Content_cut'].values))

In [ ]:
plt.figure(figsize=(14, 12))
plt.imshow(wc, interpolation='bilinear')
plt.axis('off')
plt.show()

### 图片统计
在训练数据（微博）中，有些是有图片的，有些是没有图片的。

我们做一个简单统计：

In [ ]:
train_labeled_copy['Pic_Length'] = train_labeled_copy['Pictures'].apply(lambda x: len(eval(x)))
test_copy['Pic_Length'] = test_copy['Pictures'].apply(lambda x: len(eval(x)))

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 8))
ax1.set_xlim(0, 9)
ax2.set_xlim(0, 9)

sns.distplot(train_labeled_copy['Pic_Length'], bins=25, ax=ax1, color='blue', kde=False)
sns.distplot(test_copy['Pic_Length'], bins=25, ax=ax2, color='green', kde=False)

plt.rcParams['font.sans-serif']=['SimHei']
plt.rcParams['axes.unicode_minus'] = False

ax1.set_title('训练集图片数量分布')
ax2.set_title('测试集图片数量分布')

plt.show()

可以看出分布非常近似，这里基本没什么问题。

大多数人都是不发图片或者发一张图片。

至于9图比7、8图的多，6图比5图多，大概是强迫症...

### 视频统计

视频计数在训练集和测试集中分布：

In [ ]:
train_labeled_copy['With_Video'] = train_labeled_copy['Videos'].apply(lambda x: len(eval(x)))
test_copy['With_Video'] = test_copy['Videos'].apply(lambda x: len(eval(x)))

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 8))

sns.countplot(train_labeled_copy['With_Video'], ax=ax1, color='grey')
sns.countplot(test_copy['With_Video'], ax=ax2, color='orange')

plt.rcParams['font.sans-serif']=['SimHei']
plt.rcParams['axes.unicode_minus'] = False

ax1.set_title('训练集视频分布')
ax2.set_title('测试集视频分布')

plt.show()

看起来分布是一致的。

接下来我们看看带视频和不带视频的情感标签分布。

In [ ]:
train_labeled_copy_2 = train_labeled_copy[(train_labeled_copy["Labels"] == '0') |
                                          (train_labeled_copy["Labels"] == '1') |
                                          (train_labeled_copy["Labels"] == '-1')]

sns.countplot(x='With_Video', hue='Labels', data= train_labeled_copy_2,
              order = train_labeled_copy['With_Video'].dropna().value_counts().index)
plt.show()

可以看出，大部分带视频的微博，其情感为中性。

但是明显的，相对于不带视频的微博，带视频的微博中 `正面情感` 比例比 `负面情感` 的比例更高。
